# 😀/😞 Sentiment Analysis on Restaurant Reviews using NLP

*This note book builds an end-to-end classification model uisng NLP, Naive Bayes.*

### 1.Problem

Identify whether the reviews of a restaurant made by customers are postive or negative.

### 2.Data
The data is collected from

### 3.Evaluation
Evaluation is done on basis of metrics like accuracy score.

### 4.Features
Some information about data:
* Entire data is in text format, so its better to use NLP for developing our ML models.
* Overall, we have 17000 reviews (combining both the datasets).
* We have two labels(columns) namely:
  * Review : which is the review itself.
  * Liked : which tells whether the review is positive or negative.
    * 1 - for positive
    * 0 - for negative
* Text language is english.

## Connect Google Drive to Google Colab

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

MessageError: ignored

## Get your workspace ready

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import nltk
import re
import pickle

# Other imports would be imported at specific necessary cells

## Data Preprocessing

We will do the pre-processing in three steps:
  * Process data in dataset 1
  * Process data in dataset 2
  * Combine both the datasets

Preprocessing Dataset 1

## Clean the data
  * Remove all the special characters
  * Change all the characters to lower case
  * Tokenize each reviews into words
  * Remove stopwords
  * Perform Stemming / Lemmatization

In [ ]:

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# load the final dataset into a pandas dataframe
df1 = pd.read_csv("./drive/My Drive/MLstart/financial-sentiment-analysis/giftv1/model/dfAll.csv")


In [ ]:
df = df1[["Review","Sentiment"]]


In [ ]:
df.head()

In [ ]:

# Import label encoder
from sklearn import preprocessing

# label_encoder object knows how to understand word labels.
label_encoder = preprocessing.LabelEncoder()

# Encode labels in column 'Sentiment'.
df['Sentiment']= label_encoder.fit_transform(df['Sentiment'])

df['Sentiment'].unique()

In [ ]:
df.head()

In [ ]:
# from google.colab import files
# df.to_csv("./drive/My Drive/MLstart/financial-sentiment-analysis/giftv1/dfc.csv", index = False)
# files.download('./drive/My Drive/MLstart/financial-sentiment-analysis/giftv1/dfc.csv')

In [ ]:
import nltk
nltk.download('stopwords')

In [ ]:
# Import other stopwords and PortetStemmer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

ps = PorterStemmer()

# Function for cleaning the reviews
def clean_text(text):

  # cleaning special characters from reviews
  review = re.sub(pattern='[^a-zA-Z]', repl=' ', string=text)

  # converting each review into lowercase
  review = review.lower()

  # tokenizing the review into words
  review_words = review.split()

  # removing the stop words
  review_words = [word for word in review_words if word not in set(stopwords.words('english'))]

  # stemming the words
  review_words = [ps.stem(word) for word in review_words]

  # joining the stemmed words
  review = ' '.join(review_words)

  return review


# clean text data
df["Review_cleaned"] = df["Review"].apply(lambda x: clean_text(x))

In [ ]:
df.head()

## Feature Engineering

We use vader, which uses lexicon of words to find which ones are positive and negative. It takes into account the context of the sentences and determine the below four scores.
  * a neutrality score
  * a positivity score
  * a negativity score
  * an overall score that summarizes the previous scores

We will integrate these four values as features in our dataset.

In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

sia = SentimentIntensityAnalyzer()
df['sentiments_tmp'] = df["Review"].apply(lambda x: sia.polarity_scores(x))
df = pd.concat([df.drop(['sentiments_tmp'], axis=1), df['sentiments_tmp'].apply(pd.Series)], axis=1)

Next we add some metrics to each review
* Number of characters in each review
* Number of words in each review

In [ ]:
# add number of characters column
df["nb_chars"] = df['Review'].apply(lambda x : len(x))

# add number of words column
df["nb_words"] = df["Review"].apply(lambda x : len(x.split(" ")))

In [ ]:
df.head()

Extracting vector representations for each review

In [ ]:
# create doc2Vec vector columns
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(df["Review_cleaned"].apply(lambda x: x.split(" ")))]

# train doc2vec model with our text data
model = Doc2Vec(documents, vector_size=5, window=2, min_count=1, workers=4)

# transform each document into a vector data
doc2vec_df = df["Review_cleaned"].apply(lambda x: model.infer_vector(x.split(" "))).apply(pd.Series)
doc2vec_df.columns = ["doc2vec_vector_" + str(x) for x in doc2vec_df.columns]
df = pd.concat([df, doc2vec_df], axis=1)

In [ ]:
 # save the trained doc2vec model
pickle.dump(model, open('./drive/My Drive/MLstart/financial-sentiment-analysis/giftv1/model/doc2vec-model.pkl', 'wb'))


Add TF-IDF values for every word and every document

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(min_df=10)
tfidf_result = tfidf.fit_transform(df['Review_cleaned']).toarray()
tfidf_df = pd.DataFrame(tfidf_result, columns=tfidf.get_feature_names())
tfidf_df.columns = ["word_" + str(x) for x in tfidf_df.columns]
tfidf_df.index = df.index
df = pd.concat([df, tfidf_df], axis=1)

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
# save the trained tfidf model
pickle.dump(model, open('./drive/My Drive/MLstart/financial-sentiment-analysis/giftv1/model/tfidf-model.pkl', 'wb'))



## Exploratory Data Analysis

### Word Cloud

In [ ]:
# wordcloud function

from wordcloud import WordCloud
import matplotlib.pyplot as plt

def show_wordcloud(data, title = None):
    wordcloud = WordCloud(
        background_color = 'white',
        max_words = 500,
        max_font_size = 40,
        scale = 3,
        random_state = 12
    ).generate(str(data))

    fig = plt.figure(1, figsize = (10, 10))
    plt.axis('off')
    if title:
        fig.suptitle(title, fontsize = 20)
        fig.subplots_adjust(top = 2.3)

    plt.imshow(wordcloud)
    plt.show()

# print wordcloud
show_wordcloud(df["Review"])

NameError: ignored

### Create test and train sets


In [ ]:
# feature selection
label = 'Sentiment'
ignore_columns = [label, 'Tweets', 'Review_cleaned']

features = [c for c in df.columns if not c in ignore_columns]

# split the data into test and train sets
from sklearn.model_selection import train_test_split
X, y = df[features], df[label]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)

In [ ]:
(X_train.shape, y_train.shape)

In [ ]:
(X_test.shape, y_test.shape)

### Model Building

Here we are using Naive Bayes classifier model, it works on probability.

In [ ]:
# import Random Forest Classifier from sklearn
from sklearn.ensemble import RandomForestClassifier

# create an instance for MultinomialNB
rf = RandomForestClassifier(random_state=42)

# Fit the data to the model
rf.fit(X_train, y_train)

In [ ]:
## Feature Importance

feature_importances_df = pd.DataFrame({'feature':features, 'importance':rf.feature_importances_}).sort_values('importance', ascending=False)
feature_importances_df.head(20)

### Predict test results

In [ ]:
y_preds = rf.predict(X_test)

### Evaluation

In [ ]:
# Calculate Accuracy, Precision and Recall
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

accuracy = accuracy_score(y_test, y_preds)
recall = recall_score(y_test, y_preds)
precision = precision_score(y_test, y_preds)

print("///---Scores---///")
print('Accuracy Score is : {}%'.format(round(accuracy*100,2)))
print('Precision Score is : {}'.format(round(precision,2)))
print('Recall Score is : {}'.format(round(recall,2)))

Calculate evaluation metrics with cross_val_score

In [ ]:
# import cross_val_score from sklearn
from sklearn.model_selection import cross_val_score

# Create a new classifier
classifier = RandomForestClassifier()

# Cross-Validated Accuracy
accuracy_cv = np.mean(cross_val_score(classifier,
                                      X,
                                      y,
                                      cv=5,
                                      scoring='accuracy'))

# Cross-Validated Precision
precision_cv = np.mean(cross_val_score(classifier,
                                      X,
                                      y,
                                      cv=5,
                                      scoring='precision'))

# Cross-Validated Recall
recall_cv = np.mean(cross_val_score(classifier,
                                      X,
                                      y,
                                      cv=5,
                                      scoring='recall'))

print("///---Cross-Validated Scores---///")
print('Accuracy Score is : {}%'.format(round(accuracy_cv*100,2)))
print('Precision Score is : {}'.format(round(precision_cv,2)))
print('Recall Score is : {}'.format(round(recall_cv,2)))

### Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_preds)
cm

In [ ]:
# Plotting confusion matrix
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

plt.figure(figsize=(6,6))
sns.heatmap(cm, annot=True, cmap="YlGnBu", xticklabels=['Neagtive', 'Positive'], yticklabels=['Negative', 'Positive'])
plt.xlabel('Predicted Values')
plt.ylabel('Actual Values')

### Hyperparameter Tuning using RandomizedSearchCV

In [ ]:
# # import RandomizedSearchCV from sklearn
# from sklearn.model_selection import RandomizedSearchCV

# # Create a hyperparameter grid for our classifier
# param_grid = {'alpha':np.arange(0.1,1.1,0.1)}

# # Setup hyperparameter search for RandomForestClassifier classifier
# rs = RandomizedSearchCV(RandomForestClassifier(),
#                         param_distributions=param_grid,
#                         cv=5,
#                         n_iter=10,
#                         verbose=True,
#                         random_state=0)

# # Fit the hyperparameter search
# rs.fit(X_train ,y_train)


In [ ]:
# rs.best_params_

In [ ]:
# rs_y_preds = rs.predict(X_test)
# print(classification_report(y_test, rs_y_preds))

In [ ]:
# rs.score(X_test, y_test)

In [ ]:
# Re-train the classifier with best params derived from Hyperparameter Tuning.
classifier = RandomForestClassifier(random_state=42)
classifier.fit(X_train, y_train)

In [ ]:
# save the trained classifier to use it predictions anywhere else
# pickle.dump(classifier, open('/content/drive/My Drive/Restaurant Reviews/restaurant-sentiment-rf-model.pkl', 'wb'))
pickle.dump(classifier, open('./drive/My Drive/MLstart/financial-sentiment-analysis/v/restaurant-sentiment-rf-model.pkl', 'wb'))


## Making predictions for real-world reviews

Create a function which will take a sample review as input and give prediction as output using our trained classifier

In [ ]:
import re

In [ ]:
def predict_sentiment(sample_review):

  sample_review_cleand = re.sub(pattern='[^a-zA-Z]', repl=' ', string=sample_review)
  sample_review_cleand = sample_review_cleand.lower()
  sample_review_words = sample_review_cleand.split()
  sample_review_words = [word for word in sample_review_words if word not in set(stopwords.words('english'))]
  ps = PorterStemmer()
  sample_review_words = [ps.stem(word) for word in sample_review_words]
  sample_review_cleand = ' '.join(sample_review_words)
  sample_df = pd.DataFrame()
  sample_df['Review'] = [sample_review]
  sample_df['Review_cleaned'] = [sample_review_cleand]
  sample_df['sentiments_tmp'] = sample_df["Review"].apply(lambda x: sia.polarity_scores(x))
  sample_df = pd.concat([sample_df.drop(['sentiments_tmp'], axis=1), sample_df['sentiments_tmp'].apply(pd.Series)], axis=1)
  sample_df["nb_chars"] = sample_df['Review'].apply(lambda x : len(x))
  sample_df["nb_words"] = sample_df["Review"].apply(lambda x : len(x.split(" ")))
  sample_doc2vec_df = sample_df["Review_cleaned"].apply(lambda x: model.infer_vector(x.split(" "))).apply(pd.Series)
  sample_doc2vec_df.columns = ["doc2vec_vector_" + str(x) for x in sample_doc2vec_df.columns]
  sample_df = pd.concat([sample_df, sample_doc2vec_df], axis=1)
  sample_tfidf_result = tfidf.transform(sample_df['Review_cleaned']).toarray()
  sample_tfidf_df = pd.DataFrame(sample_tfidf_result, columns=tfidf.get_feature_names())
  sample_tfidf_df.columns = ["word_" + str(x) for x in sample_tfidf_df.columns]
  sample_tfidf_df.index = sample_df.index
  sample_df = pd.concat([sample_df, sample_tfidf_df], axis=1)
  sample_features = [c for c in sample_df.columns if not c in ['Review', 'Review_cleaned']]

  return classifier.predict(sample_df[sample_features])

In [ ]:
# Predicting values
sample_review = 'The poor banking skills'
print(type(predict_sentiment(sample_review)[0]))

if predict_sentiment(sample_review) == 1:
  print('This is a POSITIVE REVIEW')
else:
  print('This is NEGATIVE REVIEW')


In [ ]:
# Predicting values
sample_review = 'Food was pretty bad and the service was very slow.'

if predict_sentiment(sample_review) == 1:
  print('This is a POSITIVE REVIEW')
else:
  print('This is NEGATIVE REVIEW')

In [ ]:
# Predicting values
sample_review = 'The food was absolutely wonderful, from preparation to presentation, very pleasing.'

if predict_sentiment(sample_review) == 1:
  print('This is a POSITIVE REVIEW')
else:
  print('This is NEGATIVE REVIEW')

In [ ]:
# Predicting values
sample_review = 'I am not Happy with food .'

if predict_sentiment(sample_review) == 1:
  print('This is a POSITIVE REVIEW')
else:
  print('This is NEGATIVE REVIEW')